In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
def plot_diversity(value_x, value_y, name):
    plt.figure()
    plt.plot(value_x, value_y, marker='o', linestyle='-', color='b')

    # Add title and labels
    plt.title('Porcentaje de diversidad (población nueva) para entrenamiento')
    plt.xlabel('Iteraciones')
    plt.ylabel('Porcentaje')
    plt.xticks(value_x)


    # show grid
    plt.grid(True)

    # save plot
    plt.savefig(name)
    plt.close()

In [7]:
def read_dir_plot(input_dir):
    for dirname, _, filenames in os.walk(input_dir):
        for filename in filenames:
            try:
                # Linux filesystem
                algorithm, problem = dirname.split("/")[-2:]
            except ValueError:
                # Windows filesystem
                algorithm, problem = dirname.split("\\")[-2:]

            if "DIV" in filename:
                run_tag = [s for s in filename.split(".") if s.isdigit()].pop()

                content = []
                with open(os.path.join(dirname, filename), "r") as content_file:
                    for line in content_file:
                        line = line.replace("[", "")
                        line = line.replace("]", "")
                        line = line.replace(",", "")
                        vector = [float(x) for x in line.split()]
                        content.append(vector)
                    div_index = list()
                    div_result = list()
                    for i in range(len(content)):
                        div_index.append(i+1)
                        div_result.append(content[i][1]/content[i][0])
                    plot_diversity(div_index, div_result, os.path.join(dirname, "diversity_" + run_tag + "_" + algorithm + "_"+ problem +".png"))
                

In [8]:
read_dir_plot("data_zdt")